In [3]:
from pandas import Series, DataFrame
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Adoption Data

In [4]:
adopt = pd.read_csv('aac_intakes_outcomes.csv')
adopt['outcome_type'].value_counts()

Adoption           33594
Transfer           23799
Return to Owner    14791
Euthanasia          6244
Died                 690
Disposal             304
Rto-Adopt            179
Missing               46
Relocate              15
Name: outcome_type, dtype: int64

In [5]:
adopt.describe

<bound method DataFrame.describe of       age_upon_outcome animal_id_outcome  date_of_birth outcome_subtype  \
0             10 years           A006100    7/9/07 0:00             NaN   
1              7 years           A006100    7/9/07 0:00             NaN   
2              6 years           A006100    7/9/07 0:00             NaN   
3             10 years           A047759    4/2/04 0:00         Partner   
4             16 years           A134067  10/16/97 0:00             NaN   
5             15 years           A141142    6/1/98 0:00             NaN   
6             15 years           A163459  10/19/99 0:00             NaN   
7             15 years           A165752   8/18/99 0:00             NaN   
8             15 years           A178569   3/17/99 0:00             NaN   
9             18 years           A189592    8/1/97 0:00             NaN   
10            16 years           A191351   8/21/99 0:00             NaN   
11            14 years           A197810   1/21/00 0:00         